In [1]:
'''
Gets posts of instagram related to savings and uploads them to S3
'''
import sys
import platform
import locale
if platform.system() == 'Linux':
    sys.path.append("/home/ubuntu/Documents/deep_dive/dd_toolkit")

import boto3
import json
import os
s3 = boto3.client('s3')

import insta_scraper as insta
from insta_listkeys import keys
import insta_dictionaries as idict
from InstagramAPI import InstagramAPI
#from tags import tag_posts

import logging
import time
from datetime import datetime

import reverse_geocoder as rg

logger = logging.getLogger(__name__)

In [2]:
def upload_s3(posts, bucket_name, folder_name):
    for p in posts:
        filename = str(p['caption']['media_id'])+'.json'
        p_json = json.dumps(p)
        f = open(filename,"w")
        f.write(p_json)
        f.close()
        
        s3.upload_file(filename, bucket_name, folder_name + filename)
        
        os.remove(filename)

In [3]:
def filter_location(posts):
    f =[]

    for p in posts:
        if "location" in p.keys():
            lat = p["location"]["lat"]
            lng = p["location"]["lng"]
            
            res = rg.get((lat, lng))
            
            if res['cc'] == 'MX':
                f.append(p)
            
    return f

In [4]:
def get_posts(API, tag):
    try:
        logger.info('Tag: {}. Getting media...'.format(tag))
        posts_items = insta.get_tag_feed(tag, API, type_ = 'items')
        logger.info('{} posts'.format(len(posts_items)))
        count_uploded_to_elastic, i = 0, 1

        filtered = posts_items
        #filtered = filter_location(posts_items)

        if len(filtered) > 0:
            logger.info('{} from Mexico'.format(len(filtered)))
        else:
            logger.info('0 from Mexico')

    except:
        logging.exception('CRITICAL ERROR uploading posts for: {}'.format(tag,str(sys.exc_info()[0])))
    
    return filtered

In [5]:
def main(tema):
    logger.info('Init dicts and database connections...')

    bucket_name = 'dd-social-networks'
    folder_name = tema+'/instagram/posts/'
    tags = ['tatuaje', 'tatuajes', 'tattoo', 'tattoos']
    
    API = InstagramAPI(keys[3]['user_name'],keys[3]['pwd'])
    API.login()
    
    for index,tag in enumerate(tags):
        try:
            posts = get_posts(API, tag)
            upload_s3(posts, bucket_name, folder_name)
            print(index,end="")
        except Exception as e:
            print('Error in tag: ', tag)
            print(e)
    
    API.logout()
    logger.info('Finished!')

In [ ]:
if __name__ == "__main__":
    format_t = "%(asctime)s [%(levelname)s] %(message)s"
    logging.basicConfig(format=format_t,level=logging.INFO)#filename='congressesMain.log')
    while True:
        main("tattoos")
        time.sleep(100)

2019-02-12 16:58:29,079 [INFO] Init dicts and database connections...
2019-02-12 16:58:29,301 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 16:58:29,820 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 16:58:31,015 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthe

Request return 429 error!
{'status': 'fail', 'message': 'Please wait a few minutes before you try again.'}


2019-02-12 16:58:33,083 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Request return 404 error!


2019-02-12 16:58:39,362 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 16:58:39,730 [INFO] Tag: tatuaje. Getting media...
2019-02-12 16:58:39,840 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



...

2019-02-15 21:49:51,404 [INFO] 84 posts
2019-02-15 21:49:51,405 [INFO] 84 from Mexico
2019-02-15 21:51:02,924 [INFO] Starting new HTTPS connection (29): dd-social-networks.s3.us-west-2.amazonaws.com
2019-02-15 21:51:39,652 [INFO] Tag: tattoos. Getting media...


Error in tag:  tattoo
'NoneType' object is not subscriptable


2019-02-15 21:51:41,715 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-15 21:51:44,650 [INFO] 79 posts
2019-02-15 21:51:44,652 [INFO] 79 from Mexico
2019-02-15 21:53:56,061 [INFO] Resetting dropped connection: dd-social-networks.s3.us-west-2.amazonaws.com
2019-02-15 21:54:03,488 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



3

2019-02-15 21:54:03,817 [INFO] Finished!
